In [2]:
import pyspark
from pyspark.sql import  SparkSession
import os

In [3]:
spark = SparkSession.builder \
    .appName('B345') \
    .getOrCreate()


In [4]:
# read csv file
df = spark.read.option('header', 'true').csv('B3_B4_processed.csv', inferSchema=True)

In [4]:
df.show()

+---+----------+----------+-----------+-------------+-----------------+-------------+-----------------+------------------+---+---+
|_c0|   _FileId|_SegmentNr|Snelheid_EW|B3_Time_Based|B3_Abs_Plaus_Time|B4_Time_Based|B4_Abs_Plaus_Time|  lengthCoordinate| B3| B4|
+---+----------+----------+-----------+-------------+-----------------+-------------+-----------------+------------------+---+---+
|  0|1651941735|         0|     6232.9|         null|             null|         null|             null|               0.0|0.0|0.0|
|  1|1651941735|         1|     6232.9|         null|             null|         null|             null|          0.249316|0.0|0.0|
|  2|1651941735|         2|     6235.6|         null|             null|         null|             null|          0.498632|0.0|0.0|
|  3|1651941735|         3|     6246.4|         null|             null|         null|             null|          0.748056|0.0|0.0|
|  4|1651941735|         4|     6246.4|         null|             null|         nul

In [5]:
# show shcema of the dataframe
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _FileId: integer (nullable = true)
 |-- _SegmentNr: integer (nullable = true)
 |-- Snelheid_EW: double (nullable = true)
 |-- B3_Time_Based: double (nullable = true)
 |-- B3_Abs_Plaus_Time: double (nullable = true)
 |-- B4_Time_Based: double (nullable = true)
 |-- B4_Abs_Plaus_Time: double (nullable = true)
 |-- lengthCoordinate: double (nullable = true)
 |-- B3: double (nullable = true)
 |-- B4: double (nullable = true)



In [5]:
df2 = df.drop('Unnamed: 0')
df3 = df2.drop('_SegmentNr', 'Snelheid_EW', 'B3_Time_Based', 'B3_Abs_Plaus_Time', 'B4_Time_Based', 'B4_Abs_Plaus_Time')
df4 = df3.drop('_c0')

In [11]:
df4.show()

+----------+------------------+---+---+
|   _FileId|  lengthCoordinate| B3| B4|
+----------+------------------+---+---+
|1651941735|               0.0|0.0|0.0|
|1651941735|          0.249316|0.0|0.0|
|1651941735|          0.498632|0.0|0.0|
|1651941735|          0.748056|0.0|0.0|
|1651941735|          0.997912|0.0|0.0|
|1651941735|          1.247768|0.0|0.0|
|1651941735|          1.497408|0.0|0.0|
|1651941735|1.7468320000000002|0.0|0.0|
|1651941735|1.9960400000000003|0.0|0.0|
|1651941735|          2.245788|0.0|0.0|
|1651941735|          2.495536|0.0|0.0|
|1651941735|           2.74496|0.0|0.0|
|1651941735|2.9942759999999997|0.0|0.0|
|1651941735|3.2440239999999996|0.0|0.0|
|1651941735|3.4940959999999994|0.0|0.0|
|1651941735|3.7438439999999993|0.0|0.0|
|1651941735| 3.993483999999999|0.0|0.0|
|1651941735| 4.242907999999999|0.0|0.0|
|1651941735| 4.492547999999999|0.0|0.0|
|1651941735|          4.741972|0.0|0.0|
+----------+------------------+---+---+
only showing top 20 rows



## B3

In [7]:
df_B3 = df4.drop('B4')
df_B3.printSchema()

root
 |-- _FileId: integer (nullable = true)
 |-- lengthCoordinate: double (nullable = true)
 |-- B3: double (nullable = true)



In [8]:
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.functions import col

df_B3 = df_B3 \
  .withColumn("lengthCoordinate" ,
              df_B3["lengthCoordinate"]
              .cast(FloatType()))   \
  .withColumn("B3",
              df_B3["B3"]
              .cast(FloatType()))

df_B3.show()

+----------+----------------+---+
|   _FileId|lengthCoordinate| B3|
+----------+----------------+---+
|1651941735|             0.0|0.0|
|1651941735|        0.249316|0.0|
|1651941735|        0.498632|0.0|
|1651941735|        0.748056|0.0|
|1651941735|        0.997912|0.0|
|1651941735|        1.247768|0.0|
|1651941735|        1.497408|0.0|
|1651941735|        1.746832|0.0|
|1651941735|         1.99604|0.0|
|1651941735|        2.245788|0.0|
|1651941735|        2.495536|0.0|
|1651941735|         2.74496|0.0|
|1651941735|        2.994276|0.0|
|1651941735|        3.244024|0.0|
|1651941735|        3.494096|0.0|
|1651941735|        3.743844|0.0|
|1651941735|        3.993484|0.0|
|1651941735|        4.242908|0.0|
|1651941735|        4.492548|0.0|
|1651941735|        4.741972|0.0|
+----------+----------------+---+
only showing top 20 rows



## add coilNumber to B3

In [9]:
# read coilInfo CSV file
df_coil_b3 = spark.read.option('header', 'true').csv('referentieB3_B4.csv', inferSchema=True)

In [10]:
df_coil_b3 = df_coil_b3.drop('_TimeStamp')
df_coil_b3.show()

+----------+---------+
|   _FileId|Rolnummer|
+----------+---------+
|1652730130|   283798|
|1653018826|   283800|
|1653535200|   283801|
|1653015786|   283802|
|1653509472|   283803|
|1651941735|   283804|
|1652492652|   283805|
|1652986410|   283806|
|1652442547|   283807|
|1652983516|   283808|
|1652434323|   283809|
|1653209077|   283810|
|1652678559|   283811|
|1653181616|   283812|
|1653691585|   283813|
|1653174096|   283814|
|1653674288|   283815|
|1653116649|   283816|
|1654683836|   283817|
|1655173668|   283818|
+----------+---------+
only showing top 20 rows



In [11]:
# join rolnummer from coilInfo file to df_B3
df_b3_with_coil = df_B3.join(df_coil_b3, df_B3._FileId == df_coil_b3._FileId, how='right')
df_b3_with_coil.show()

+----------+----------------+---+----------+---------+
|   _FileId|lengthCoordinate| B3|   _FileId|Rolnummer|
+----------+----------------+---+----------+---------+
|1653116649|             0.0|0.0|1653116649|   283816|
|1653116649|        0.251072|0.0|1653116649|   283816|
|1653116649|        0.501928|0.0|1653116649|   283816|
|1653116649|        0.753216|0.0|1653116649|   283816|
|1653116649|        1.004072|0.0|1653116649|   283816|
|1653116649|        1.255144|0.0|1653116649|   283816|
|1653116649|        1.506324|0.0|1653116649|   283816|
|1653116649|        1.757828|0.0|1653116649|   283816|
|1653116649|         2.00944|0.0|1653116649|   283816|
|1653116649|        2.260404|0.0|1653116649|   283816|
|1653116649|        2.511152|0.0|1653116649|   283816|
|1653116649|          2.7619|0.0|1653116649|   283816|
|1653116649|         3.01308|0.0|1653116649|   283816|
|1653116649|        3.264152|0.0|1653116649|   283816|
|1653116649|        3.515116|0.0|1653116649|   283816|
|165311664

In [12]:
df_b3_with_coil = df_b3_with_coil.drop('_FileId')

In [13]:
# rename column
df_b3_with_coil = df_b3_with_coil.withColumnRenamed('Rolnummer', 'coilId')
df_b3_with_coil.show()

+----------------+---+------+
|lengthCoordinate| B3|coilId|
+----------------+---+------+
|             0.0|0.0|283816|
|        0.251072|0.0|283816|
|        0.501928|0.0|283816|
|        0.753216|0.0|283816|
|        1.004072|0.0|283816|
|        1.255144|0.0|283816|
|        1.506324|0.0|283816|
|        1.757828|0.0|283816|
|         2.00944|0.0|283816|
|        2.260404|0.0|283816|
|        2.511152|0.0|283816|
|          2.7619|0.0|283816|
|         3.01308|0.0|283816|
|        3.264152|0.0|283816|
|        3.515116|0.0|283816|
|        3.765864|0.0|283816|
|        4.017044|0.0|283816|
|         4.26844|0.0|283816|
|        4.519728|0.0|283816|
|        4.770584|0.0|283816|
+----------------+---+------+
only showing top 20 rows



In [14]:
# change datatype to integer
df_b3_with_coil = df_b3_with_coil \
  .withColumn("coilId" ,
              df_b3_with_coil["coilId"]
              .cast(IntegerType())) 

In [19]:
df_b3_with_coil = df_b3_with_coil.select('coilId', 'lengthCoordinate', 'B3')
df_b3_with_coil.show()

+------+----------------+---+
|coilId|lengthCoordinate| B3|
+------+----------------+---+
|283816|             0.0|0.0|
|283816|        0.251072|0.0|
|283816|        0.501928|0.0|
|283816|        0.753216|0.0|
|283816|        1.004072|0.0|
|283816|        1.255144|0.0|
|283816|        1.506324|0.0|
|283816|        1.757828|0.0|
|283816|         2.00944|0.0|
|283816|        2.260404|0.0|
|283816|        2.511152|0.0|
|283816|          2.7619|0.0|
|283816|         3.01308|0.0|
|283816|        3.264152|0.0|
|283816|        3.515116|0.0|
|283816|        3.765864|0.0|
|283816|        4.017044|0.0|
|283816|         4.26844|0.0|
|283816|        4.519728|0.0|
|283816|        4.770584|0.0|
+------+----------------+---+
only showing top 20 rows



In [19]:
df_b3_with_coil.printSchema()

root
 |-- coilId: integer (nullable = true)
 |-- lengthCoordinate: float (nullable = true)
 |-- B3: float (nullable = true)



In [ ]:
# write dataframe to database
url = "jdbc:sqlite:database_B345.db"
df_b3_with_coil.write.jdbc(url=url, driver='org.sqlite.JDBC', table='B3')

## B4

In [ ]:
df_B4 = df4.drop('B3')
df_B4.printSchema()

root
 |-- _FileId: integer (nullable = true)
 |-- lengthCoordinate: double (nullable = true)
 |-- B4: double (nullable = true)



In [ ]:
df_B4 = df_B4 \
  .withColumn("lengthCoordinate" ,
              df_B4["lengthCoordinate"]
              .cast(FloatType()))   \
  .withColumn("B4",
              df_B4["B4"]
              .cast(FloatType()))

df_B4.printSchema()

root
 |-- _FileId: integer (nullable = true)
 |-- lengthCoordinate: float (nullable = true)
 |-- B4: float (nullable = true)



In [ ]:
df_b4_with_coil = df_B4.join(df_coil_b3, df_B4._FileId == df_coil_b3._FileId, how='left')
df_b4_with_coil.show()

+----------+----------------+---+----------+---------+
|   _FileId|lengthCoordinate| B4|   _FileId|Rolnummer|
+----------+----------------+---+----------+---------+
|1651941735|             0.0|0.0|1651941735|   283804|
|1651941735|        0.249316|0.0|1651941735|   283804|
|1651941735|        0.498632|0.0|1651941735|   283804|
|1651941735|        0.748056|0.0|1651941735|   283804|
|1651941735|        0.997912|0.0|1651941735|   283804|
|1651941735|        1.247768|0.0|1651941735|   283804|
|1651941735|        1.497408|0.0|1651941735|   283804|
|1651941735|        1.746832|0.0|1651941735|   283804|
|1651941735|         1.99604|0.0|1651941735|   283804|
|1651941735|        2.245788|0.0|1651941735|   283804|
|1651941735|        2.495536|0.0|1651941735|   283804|
|1651941735|         2.74496|0.0|1651941735|   283804|
|1651941735|        2.994276|0.0|1651941735|   283804|
|1651941735|        3.244024|0.0|1651941735|   283804|
|1651941735|        3.494096|0.0|1651941735|   283804|
|165194173

In [ ]:
df_b4_with_coil = df_b4_with_coil.drop('_FileId')

df_b4_with_coil = df_b4_with_coil.withColumnRenamed('Rolnummer', 'coilId')
df_b4_with_coil.show()


+----------------+---+------+
|lengthCoordinate| B4|coilId|
+----------------+---+------+
|             0.0|0.0|283804|
|        0.249316|0.0|283804|
|        0.498632|0.0|283804|
|        0.748056|0.0|283804|
|        0.997912|0.0|283804|
|        1.247768|0.0|283804|
|        1.497408|0.0|283804|
|        1.746832|0.0|283804|
|         1.99604|0.0|283804|
|        2.245788|0.0|283804|
|        2.495536|0.0|283804|
|         2.74496|0.0|283804|
|        2.994276|0.0|283804|
|        3.244024|0.0|283804|
|        3.494096|0.0|283804|
|        3.743844|0.0|283804|
|        3.993484|0.0|283804|
|        4.242908|0.0|283804|
|        4.492548|0.0|283804|
|        4.741972|0.0|283804|
+----------------+---+------+
only showing top 20 rows



In [ ]:
df_b4_with_coil = df_b4_with_coil \
  .withColumn("coilId" ,
              df_b4_with_coil["coilId"]
              .cast(IntegerType())) 

In [ ]:
df_b4_with_coil = df_b4_with_coil.select('coilId', 'lengthCoordinate', 'B4')

In [ ]:
df_b4_with_coil.show()

+------+----------------+---+
|coilId|lengthCoordinate| B4|
+------+----------------+---+
|283804|             0.0|0.0|
|283804|        0.249316|0.0|
|283804|        0.498632|0.0|
|283804|        0.748056|0.0|
|283804|        0.997912|0.0|
|283804|        1.247768|0.0|
|283804|        1.497408|0.0|
|283804|        1.746832|0.0|
|283804|         1.99604|0.0|
|283804|        2.245788|0.0|
|283804|        2.495536|0.0|
|283804|         2.74496|0.0|
|283804|        2.994276|0.0|
|283804|        3.244024|0.0|
|283804|        3.494096|0.0|
|283804|        3.743844|0.0|
|283804|        3.993484|0.0|
|283804|        4.242908|0.0|
|283804|        4.492548|0.0|
|283804|        4.741972|0.0|
+------+----------------+---+
only showing top 20 rows



In [ ]:
# write dataframe B4 to database 
url = "jdbc:sqlite:database_B345.db"
df_b4_with_coil.write.jdbc(url=url, driver='org.sqlite.JDBC', table='B4')

## dataframe B5

In [8]:
df_B5 = spark.read.option('header', 'true').csv('B5_processed.csv', inferSchema=True)
df_B5.show()

+---+----------+---+------------------+
|_c0|   _FileId| B5|  lengthCoordinate|
+---+----------+---+------------------+
|  0|1651763725|0.0|               0.0|
|  1|1651763725|0.0|         0.2920536|
|  2|1651763725|0.0|         0.5843232|
|  3|1651763725|0.0|0.8767872000000001|
|  4|1651763725|0.0|1.1692620000000002|
|  5|1651763725|0.0|1.4618016000000003|
|  6|1651763725|0.0|1.7540496000000003|
|  7|1651763725|0.0|2.0462976000000004|
|  8|1651763725|0.0|2.3385888000000006|
|  9|1651763725|0.0|2.6312688000000004|
| 10|1651763725|0.0|2.9239488000000002|
| 11|1651763725|0.0|         3.2166396|
| 12|1651763725|0.0|         3.5094276|
| 13|1651763725|0.0|         3.8021616|
| 14|1651763725|0.0|         4.0943988|
| 15|1651763725|0.0|4.3865495999999995|
| 16|1651763725|0.0|         4.6788624|
| 17|1651763725|0.0|         4.9717584|
| 18|1651763725|0.0|          5.264676|
| 19|1651763725|0.0| 5.557777199999999|
+---+----------+---+------------------+
only showing top 20 rows



In [9]:
df_coil_b5 = spark.read.option('header', 'true').csv('referentieB5.csv', inferSchema=True)
df_coil_b5.show()

+----------+-------------------+---------+
|   _FileId|         _TimeStamp|Rolnummer|
+----------+-------------------+---------+
|1652013153|2022-07-19 05:49:01|   283798|
|1652296902|2022-07-19 05:53:33|   283800|
|1651763725|2022-07-19 05:54:46|   283801|
|1653342091|2022-07-19 05:56:51|   283802|
|1653833331|2022-07-19 05:58:14|   283803|
|1653319126|2022-07-19 06:00:02|   283804|
|1653869909|2022-07-19 06:02:10|   283805|
|1653316797|2022-07-19 06:03:33|   283806|
|1653820249|2022-07-19 06:04:55|   283807|
|1654364325|2022-07-19 06:06:45|   283808|
|1653808728|2022-07-19 06:08:14|   283809|
|1654584287|2022-07-19 06:09:45|   283810|
|1654053353|2022-07-19 06:11:22|   283811|
|1654564234|2022-07-19 06:12:58|   283812|
|1654023411|2022-07-19 06:14:29|   283813|
|1654551125|2022-07-19 06:16:09|   283814|
|1653999334|2022-07-19 06:17:37|   283815|
|1653442318|2022-07-19 06:19:07|   283816|
|1653959981|2022-07-19 06:20:34|   283817|
|1653399845|2022-07-19 06:22:07|   283818|
+----------

In [10]:
df_b5_with_coil = df_B5.join(df_coil_b5, df_B5._FileId == df_coil_b5._FileId, how='right')
df_b5_with_coil.show()

+---+----------+---+------------------+----------+-------------------+---------+
|_c0|   _FileId| B5|  lengthCoordinate|   _FileId|         _TimeStamp|Rolnummer|
+---+----------+---+------------------+----------+-------------------+---------+
|  0|1651763725|0.0|               0.0|1651763725|2022-07-19 05:54:46|   283801|
|  1|1651763725|0.0|         0.2920536|1651763725|2022-07-19 05:54:46|   283801|
|  2|1651763725|0.0|         0.5843232|1651763725|2022-07-19 05:54:46|   283801|
|  3|1651763725|0.0|0.8767872000000001|1651763725|2022-07-19 05:54:46|   283801|
|  4|1651763725|0.0|1.1692620000000002|1651763725|2022-07-19 05:54:46|   283801|
|  5|1651763725|0.0|1.4618016000000003|1651763725|2022-07-19 05:54:46|   283801|
|  6|1651763725|0.0|1.7540496000000003|1651763725|2022-07-19 05:54:46|   283801|
|  7|1651763725|0.0|2.0462976000000004|1651763725|2022-07-19 05:54:46|   283801|
|  8|1651763725|0.0|2.3385888000000006|1651763725|2022-07-19 05:54:46|   283801|
|  9|1651763725|0.0|2.631268

In [ ]:
# check the number of the nan values
from pyspark.sql.functions import col,isnan, when, count
df_b5_with_coil.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_b5_with_coil.columns]
   ).show()

In [ ]:
df_b5_with_coil = df_b5_with_coil.drop('_FileId', '_c0', '_FileId', '_TimeStamp')

df_b5_with_coil = df_b5_with_coil.withColumnRenamed('Rolnummer', 'coilId')
df_b5_with_coil.show()

+---+------------------+------+
| B5|  lengthCoordinate|coilId|
+---+------------------+------+
|0.0|               0.0|283801|
|0.0|         0.2920536|283801|
|0.0|         0.5843232|283801|
|0.0|0.8767872000000001|283801|
|0.0|1.1692620000000002|283801|
|0.0|1.4618016000000003|283801|
|0.0|1.7540496000000003|283801|
|0.0|2.0462976000000004|283801|
|0.0|2.3385888000000006|283801|
|0.0|2.6312688000000004|283801|
|0.0|2.9239488000000002|283801|
|0.0|         3.2166396|283801|
|0.0|         3.5094276|283801|
|0.0|         3.8021616|283801|
|0.0|         4.0943988|283801|
|0.0|4.3865495999999995|283801|
|0.0|         4.6788624|283801|
|0.0|         4.9717584|283801|
|0.0|          5.264676|283801|
|0.0| 5.557777199999999|283801|
+---+------------------+------+
only showing top 20 rows



In [ ]:
df_b5_with_coil = df_b5_with_coil \
  .withColumn("coilId" ,
              df_b5_with_coil["coilId"]
              .cast(IntegerType())).withColumn("B5",
              df_b5_with_coil["B5"]
              .cast(FloatType()))

In [ ]:
df_b5_with_coil = df_b5_with_coil.select('coilId', 'lengthCoordinate', 'B5')

In [ ]:
df_b5_with_coil.show()

+------+------------------+---+
|coilId|  lengthCoordinate| B5|
+------+------------------+---+
|283801|               0.0|0.0|
|283801|         0.2920536|0.0|
|283801|         0.5843232|0.0|
|283801|0.8767872000000001|0.0|
|283801|1.1692620000000002|0.0|
|283801|1.4618016000000003|0.0|
|283801|1.7540496000000003|0.0|
|283801|2.0462976000000004|0.0|
|283801|2.3385888000000006|0.0|
|283801|2.6312688000000004|0.0|
|283801|2.9239488000000002|0.0|
|283801|         3.2166396|0.0|
|283801|         3.5094276|0.0|
|283801|         3.8021616|0.0|
|283801|         4.0943988|0.0|
|283801|4.3865495999999995|0.0|
|283801|         4.6788624|0.0|
|283801|         4.9717584|0.0|
|283801|          5.264676|0.0|
|283801| 5.557777199999999|0.0|
+------+------------------+---+
only showing top 20 rows



In [ ]:
# write dataframe B5 to database 
url = "jdbc:sqlite:database_B345.db"
df_b4_with_coil.write.jdbc(url=url, driver='org.sqlite.JDBC', table='B5')

In [ ]:
import pandas as pd


In [ ]:
coilData_pandas = pd.read_excel('CoilDataInput.xlsx') 
coilData_pandas

,coil,furnace Number,analyse,Hardness_1,Hardness_2,Width,Temperature before finishing mill,Temperature after finishing mill,Thickness,Thickness profile,...,P,S,Al,M_Al,B,N,Ti,Cr,V,Mo
0,283642,4,L651,12157,122,1731.316667,0.000000,906.25,4.35100,44.337225,...,133,11,326,319,3,62,29,3469,10,18
1,283643,1,KN71,11888,124,1602.800000,0.000000,887.75,3.74800,56.353765,...,142,76,309,291,1,51,9,145,1,15
2,283644,4,KM81,12419,129,1637.133333,0.000000,878.50,3.00950,48.505748,...,190,66,343,332,1,39,11,176,0,11
3,283645,1,KM31,11106,116,1697.450000,0.000000,869.50,3.01300,38.305057,...,136,56,322,283,1,57,14,209,0,10
4,283646,4,KM31,11106,116,1693.975000,0.000000,878.50,3.01300,36.405860,...,136,56,322,283,1,57,14,209,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25298,350538,1,LM41,13043,136,766.033333,1188.085160,904.75,2.19975,29.012289,...,232,33,323,312,2,44,20,208,19,21
25299,350539,4,KM41,11473,120,821.300000,1160.497725,908.75,2.00050,26.631035,...,154,79,267,251,1,35,12,169,8,12
25300,350540,1,KM81,12368,129,775.950000,1195.974522,899.00,2.50900,34.303870,...,140,46,331,319,1,38,9,155,2,10
25301,350541,1,LM41,13027,136,769.300000,1182.732661,907.50,2.24750,27.411123,...,236,31,309,296,2,53,21,251,28,18


In [ ]:
coilData_pandas.columns

Index(['coil', 'furnace Number', 'analyse', 'Hardness_1', 'Hardness_2',
       'Width', 'Temperature before finishing mill',
       'Temperature after finishing mill', 'Thickness', 'Thickness profile',
       'C', 'Mn', 'Si', 'Nb', 'P', 'S', 'Al', 'M_Al', 'B', 'N', 'Ti', 'Cr',
       'V', 'Mo'],
      dtype='object')

In [ ]:
# conver pandas dataframe to spark dataframe
coilData_pyspark = spark.createDataFrame(coilData_pandas)

In [ ]:
coilData_pyspark.printSchema()

root
 |-- coil: long (nullable = true)
 |-- furnace Number: long (nullable = true)
 |-- analyse: string (nullable = true)
 |-- Hardness_1: long (nullable = true)
 |-- Hardness_2: long (nullable = true)
 |-- Width: double (nullable = true)
 |-- Temperature before finishing mill: double (nullable = true)
 |-- Temperature after finishing mill: double (nullable = true)
 |-- Thickness: double (nullable = true)
 |-- Thickness profile: double (nullable = true)
 |-- C: long (nullable = true)
 |-- Mn: long (nullable = true)
 |-- Si: long (nullable = true)
 |-- Nb: long (nullable = true)
 |-- P: long (nullable = true)
 |-- S: long (nullable = true)
 |-- Al: long (nullable = true)
 |-- M_Al: long (nullable = true)
 |-- B: long (nullable = true)
 |-- N: long (nullable = true)
 |-- Ti: long (nullable = true)
 |-- Cr: long (nullable = true)
 |-- V: long (nullable = true)
 |-- Mo: long (nullable = true)



In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType

# change datatype to integer
coilData_pyspark = coilData_pyspark.select([col(column).cast('integer') for column in coilData_pyspark.columns])

In [ ]:
coilData_pyspark.printSchema()

root
 |-- coil: integer (nullable = true)
 |-- furnace Number: integer (nullable = true)
 |-- analyse: integer (nullable = true)
 |-- Hardness_1: integer (nullable = true)
 |-- Hardness_2: integer (nullable = true)
 |-- Width: integer (nullable = true)
 |-- Temperature before finishing mill: integer (nullable = true)
 |-- Temperature after finishing mill: integer (nullable = true)
 |-- Thickness: integer (nullable = true)
 |-- Thickness profile: integer (nullable = true)
 |-- C: integer (nullable = true)
 |-- Mn: integer (nullable = true)
 |-- Si: integer (nullable = true)
 |-- Nb: integer (nullable = true)
 |-- P: integer (nullable = true)
 |-- S: integer (nullable = true)
 |-- Al: integer (nullable = true)
 |-- M_Al: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- N: integer (nullable = true)
 |-- Ti: integer (nullable = true)
 |-- Cr: integer (nullable = true)
 |-- V: integer (nullable = true)
 |-- Mo: integer (nullable = true)



In [ ]:

coilData_pyspark1 = coilData_pyspark \
.withColumn('analyse',col('analyse').cast('string')) \
.withColumn('Temperature after finishing mill',col('Temperature after finishing mill').cast('float')) \
.withColumn('Width',col('Width').cast('float')) \
.withColumn('Thickness',col('Thickness').cast('float')) \
.withColumn('Thickness profile',col('Thickness profile').cast('float')) \


In [ ]:
coilData_pyspark1.printSchema()

root
 |-- coil: integer (nullable = true)
 |-- furnace Number: integer (nullable = true)
 |-- analyse: string (nullable = true)
 |-- Hardness_1: integer (nullable = true)
 |-- Hardness_2: integer (nullable = true)
 |-- Width: float (nullable = true)
 |-- Temperature before finishing mill: integer (nullable = true)
 |-- Temperature after finishing mill: float (nullable = true)
 |-- Thickness: float (nullable = true)
 |-- Thickness profile: float (nullable = true)
 |-- C: integer (nullable = true)
 |-- Mn: integer (nullable = true)
 |-- Si: integer (nullable = true)
 |-- Nb: integer (nullable = true)
 |-- P: integer (nullable = true)
 |-- S: integer (nullable = true)
 |-- Al: integer (nullable = true)
 |-- M_Al: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- N: integer (nullable = true)
 |-- Ti: integer (nullable = true)
 |-- Cr: integer (nullable = true)
 |-- V: integer (nullable = true)
 |-- Mo: integer (nullable = true)



In [ ]:
coilData_pyspark2 = coilData_pyspark1.withColumnRenamed('furnace Number', 'furnace_Number')
coilData_pyspark2 = coilData_pyspark2.withColumnRenamed('Temperature before finishing mill', 'Temp_before_finishing_mill')
coilData_pyspark2 = coilData_pyspark2.withColumnRenamed('Temperature after finishing mill', 'Temp_after_finishing_mill')
coilData_pyspark2 = coilData_pyspark2.withColumnRenamed('Thickness profile', 'Thickness_profile')
coilData_pyspark2.show()

+------+--------------+-------+----------+----------+------+--------------------------+-------------------------+---------+-----------------+----+-----+----+---+---+---+---+----+---+---+---+----+---+---+
|  coil|furnace_Number|analyse|Hardness_1|Hardness_2| Width|Temp_before_finishing_mill|Temp_after_finishing_mill|Thickness|Thickness_profile|   C|   Mn|  Si| Nb|  P|  S| Al|M_Al|  B|  N| Ti|  Cr|  V| Mo|
+------+--------------+-------+----------+----------+------+--------------------------+-------------------------+---------+-----------------+----+-----+----+---+---+---+---+----+---+---+---+----+---+---+
|283642|             4|   null|     12157|       122|1731.0|                         0|                    906.0|      4.0|             44.0| 849|15470|2925| 11|133| 11|326| 319|  3| 62| 29|3469| 10| 18|
|283643|             1|   null|     11888|       124|1602.0|                         0|                    887.0|      3.0|             56.0| 704| 5340|  83|344|142| 76|309| 291|  1| 5

In [ ]:
## write dataframe coilData to database
url = "jdbc:sqlite:B345.db"
coilData_pyspark2.write.jdbc(url=url, driver='org.sqlite.JDBC', table='coilData')

# write to csv
# coilData_pyspark2.write.csv('coilData', header=True)